# Pipline Detector

In [32]:
import json
import math
from exiftool import ExifToolHelper
import cv2
import matplotlib.pyplot as plt
import numpy as np
from skimage.util import img_as_ubyte

# Image Alignment

In [33]:
# Set Image paths for testing 

image_paths = [
    r"data\example\IMG_0040_1.tif",
    r"data\example\IMG_0040_2.tif",
    r"data\example\IMG_0040_3.tif",
    r"data\example\IMG_0040_4.tif",
    r"data\example\IMG_0040_5.tif",
]

image_paths = ["../../../" + path for path in image_paths]

In [34]:
with ExifToolHelper() as et:
    for d in et.get_metadata("../../../" + image_paths[0]):
        for k, v in d.items():
            print(f"Dict: {k} = {v}")

Dict: SourceFile = ../../../data/example/IMG_0040_1.tif
Dict: ExifTool:ExifToolVersion = 12.85
Dict: File:FileName = IMG_0040_1.tif
Dict: File:Directory = ../../../data/example
Dict: File:FileSize = 3176902
Dict: File:FileModifyDate = 2024:05:18 12:10:56+02:00
Dict: File:FileAccessDate = 2024:05:22 20:44:27+02:00
Dict: File:FileCreateDate = 2024:05:17 23:32:22+02:00
Dict: File:FilePermissions = 100666
Dict: File:FileType = TIFF
Dict: File:FileTypeExtension = TIF
Dict: File:MIMEType = image/tiff
Dict: File:ExifByteOrder = II
Dict: EXIF:SubfileType = 0
Dict: EXIF:ImageWidth = 1456
Dict: EXIF:ImageHeight = 1088
Dict: EXIF:BitsPerSample = 16
Dict: EXIF:Compression = 1
Dict: EXIF:PhotometricInterpretation = 1
Dict: EXIF:Make = MicaSense
Dict: EXIF:Model = RedEdge-P
Dict: EXIF:StripOffsets = (Binary data 78 bytes, use -b option to extract)
Dict: EXIF:Orientation = 1
Dict: EXIF:SamplesPerPixel = 1
Dict: EXIF:RowsPerStrip = 100
Dict: EXIF:StripByteCounts = (Binary data 76 bytes, use -b option 

In [49]:
# Function to extract EXIF data
def extract_exif_data(image_path):
    with ExifToolHelper() as et:
        metadata = et.get_metadata(image_path)
    return metadata[0]

exif_data = []
for image_path in image_paths:
    metadata = extract_exif_data(image_path)
    exif_data.append({
        'file_path': image_path,
        'gps_latitude': metadata.get('EXIF:GPSLatitude'),
        'gps_longitude': metadata.get('EXIF:GPSLongitude'),
        'gps_altitude': metadata.get('EXIF:GPSAltitude'),
        'yaw': float(metadata.get('XMP:Yaw', 0)),  # Convert to float, default to 0 if not present
        'pitch': float(metadata.get('XMP:Pitch', 0)),  # Convert to float, default to 0 if not present
        'roll': float(metadata.get('XMP:Roll', 0)),  # Convert to float, default to 0 if not present
    })

# Function to create a rotation matrix based on yaw, pitch, and roll
def get_rotation_matrix(yaw, pitch, roll):
    # Convert degrees to radians
    yaw = np.deg2rad(yaw)
    pitch = np.deg2rad(pitch)
    roll = np.deg2rad(roll)

    # Rotation matrices around the x, y, and z axes
    Rx = np.array([[1, 0, 0],
                   [0, np.cos(roll), -np.sin(roll)],
                   [0, np.sin(roll), np.cos(roll)]])

    Ry = np.array([[np.cos(pitch), 0, np.sin(pitch)],
                   [0, 1, 0],
                   [-np.sin(pitch), 0, np.cos(pitch)]])

    Rz = np.array([[np.cos(yaw), -np.sin(yaw), 0],
                   [np.sin(yaw), np.cos(yaw), 0],
                   [0, 0, 1]])

    # Combined rotation matrix
    R = Rz @ Ry @ Rx

    return R[:2, :3]

# Function to align images
def align_images(images, exif_data):
    aligned_images = []

    for idx, image_path in enumerate(images):
        image = cv2.imread(image_path)
        data = exif_data[idx]

        # Extract orientation data
        yaw = data['yaw']
        pitch = data['pitch']
        roll = data['roll']

        # Get the rotation matrix
        M = get_rotation_matrix(yaw, pitch, roll).astype(np.float32)

        # Apply affine transformation
        aligned_image = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]))
        aligned_images.append(aligned_image)

    return aligned_images

aligned_images = align_images(image_paths, exif_data)

def overlay_images(images, alpha=0.5):
    overlay = images[0].astype(np.float32)
    for image in images[1:]:
        overlay = cv2.addWeighted(overlay, alpha, image.astype(np.float32), alpha, 0)
    overlay = cv2.normalize(overlay, None, 0, 255, cv2.NORM_MINMAX)
    return overlay.astype(np.uint8)

# Get the overlaid image
overlaid_image = overlay_images(aligned_images, alpha=1.0 / len(aligned_images))

# Save or display the overlaid image
#output_path = "overlaid_image.tif"
#cv2.imwrite(output_path, overlaid_image)
cv2.imshow("Overlaid Image", overlaid_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


#### Old Method

In [ ]:
# Function to normalize and equalize image intensity
def normalize_and_equalize_image(image):
    normalized = cv2.normalize(image, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
    equalized = cv2.equalizeHist(normalized)
    return equalized


# Load and process the reference image (the first one)
reference_image = cv2.imread(image_paths[0], cv2.IMREAD_GRAYSCALE)
reference_image_processed = normalize_and_equalize_image(reference_image)

# Initialize SIFT detector 
sift = cv2.SIFT_create(nfeatures=5000)

# Detect and compute descriptors for the reference image
keypoints_ref, descriptors_ref = sift.detectAndCompute(reference_image_processed, None)

# FLANN parameters
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
search_params = dict(checks=50)  # or pass empty dictionary

# Create FLANN based matcher object
flann = cv2.FlannBasedMatcher(index_params, search_params)


# Function to align an image to the reference image
def align_image(image_path, keypoints_ref, descriptors_ref):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Normalize and equalize the image intensity
    image_processed = normalize_and_equalize_image(image)

    # Detect and compute descriptors for the image
    keypoints, descriptors = sift.detectAndCompute(image_processed, None)

    # Match descriptors
    matches = flann.knnMatch(descriptors, descriptors_ref, k=2)

    # Apply Lowe's ratio test
    good_matches = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:  # Adjusted ratio test threshold
            good_matches.append(m)

    # Ensure we have enough good matches
    if len(good_matches) < 4:  # Minimum number of matches required for estimateAffinePartial2D
        print(f"Not enough matches found for {image_path}. Skipping alignment.")
        return None

    # Extract location of good matches
    points1 = np.zeros((len(good_matches), 2), dtype=np.float32)
    points2 = np.zeros((len(good_matches), 2), dtype=np.float32)

    for i, match in enumerate(good_matches):
        points1[i, :] = keypoints[match.queryIdx].pt
        points2[i, :] = keypoints_ref[match.trainIdx].pt

    # Find Euclidean transformation
    h, mask = cv2.estimateAffinePartial2D(points1, points2, method=cv2.RANSAC, ransacReprojThreshold=5.0)

    # Use the transformation to transform the image
    height, width = reference_image.shape
    aligned_image = cv2.warpAffine(image, h, (width, height))

    return aligned_image


# Align all images to the reference image
aligned_images = [reference_image]
for image_path in image_paths[1:]:
    aligned_image = align_image(image_path, keypoints_ref, descriptors_ref)
    if aligned_image is not None:
        aligned_images.append(aligned_image)

        # Display each aligned image
        plt.figure()
        plt.imshow(aligned_image, cmap='gray')
        plt.title(f'Aligned Image: {image_path}')
        plt.axis('off')
        plt.show()

# Combine aligned images (e.g., taking the mean of aligned images)
combined_image = np.mean(np.array(aligned_images), axis=0).astype(np.uint8)

# Save the combined image
output_path = "../../../" + 'data/example/combined_image.tif'
cv2.imwrite(output_path, combined_image)

# Display the combined image
plt.figure()
plt.imshow(combined_image, cmap='gray')
plt.title('Combined Image')
plt.axis('off')
plt.show()


## Panel Detector

In [ ]:
def calculate_panel_size_in_pixels(altitude, pixel_size, resolution, sensor_size_mm, focal_length_mm,
                                   physical_panel_size):
    """
    Calculate the expected size of an object in pixels based on camera parameters and object physical size.
    
    Parameters:
        altitude (float): Altitude in meters.
        pixel_size (float): Pixel size in meters.
        resolution (tuple): Image resolution (width, height) in pixels.
        sensor_size_mm (float): Sensor diagonal in millimeters.
        focal_length_m (float): Focal length in millimeters.
        physical_panel_size (tuple): Physical size of the object in meters (width, height).
    
    Returns:
        tuple: Expected width and height of the object in pixels.
    """
    # Convert sensor diagonal to meters
    sensor_diagonal = sensor_size_mm / 1000  # Convert mm to m
    focal_length = focal_length_mm / 1000

    # Calculate horizontal and vertical Field of View (FoV)
    fov_horizontal = 2 * math.atan(
        (sensor_diagonal / (2 * math.sqrt(1 + (resolution[0] / resolution[1]) ** 2))) / focal_length)
    fov_vertical = 2 * math.atan(
        (sensor_diagonal / (2 * math.sqrt(1 + (resolution[1] / resolution[0]) ** 2))) / focal_length)

    # Calculate scale in pixels per meter
    scale_pixels_per_meter = resolution[1] / (altitude * math.tan(fov_vertical / 2))

    # Calculate expected panel size in pixels
    panel_width_pixels = np.intp(physical_panel_size[0] * scale_pixels_per_meter)
    panel_height_pixels = np.intp(physical_panel_size[1] * scale_pixels_per_meter)

    return panel_width_pixels, panel_height_pixels

In [ ]:
def filter_by_variance(image, blur_kernel_size=15):
    """
    Apply a variance filter to an image to enhance features.
    
    Parameters:
        image (numpy.ndarray): Input image.
        blur_kernel_size (int): Size of the Gaussian blur kernel.
    
    Returns:
        numpy.ndarray: Filtered image.
    """
    blur_non = cv2.GaussianBlur(image, (blur_kernel_size, blur_kernel_size), 2)
    for i in range(10):
        blur_non = cv2.GaussianBlur(blur_non, (blur_kernel_size, blur_kernel_size), 2)
    last_blur_non = cv2.GaussianBlur(image, (blur_kernel_size, blur_kernel_size), 2)
    return cv2.equalizeHist(img_as_ubyte((last_blur_non - blur_non) ** 2))

In [ ]:
def contour_solidity(contour):
    """
    Calculate the solidity of a contour.
    
    Solidity is the ratio of contour area to convex hull area.
    
    Parameters:
    contour (array-like): Contour points.

    Returns:
    float: Solidity value between 0 and 1.
    """
    area = cv2.contourArea(contour)
    hull = cv2.convexHull(contour)
    hull_area = cv2.contourArea(hull)
    if hull_area == 0:
        return 0
    return float(area) / hull_area

In [ ]:
def scale_box(box, scale):
    """
    Scale the size of a bounding box while keeping its center fixed.
    
    Parameters:
        box (ndarray): Array of bounding box points.
        scale (float): Scaling factor.
        
    Returns:
        ndarray: Scaled bounding box points.
    """
    center = np.mean(box, axis=0)
    scaled_box = scale * (box - center) + center
    return scaled_box

In [ ]:
# Given/known Camera parameters
altitude = 16.3145  # Altitude in meters
pixel_size = 3.45e-6  # Pixel size in meters
resolution = (1456, 1088)  # Image resolution (width, height) in pixels
sensor_size_mm = 6.3  # Sensor diagonal in millimeters
focal_length_mm = 5.5  # Focal length in millimeters
physical_panel_size = (1, 1)  # Physical size of the object in meters (width, height)

panel_size_pixels = calculate_panel_size_in_pixels(
    altitude, pixel_size, resolution, sensor_size_mm, focal_length_mm, physical_panel_size
)
print("Expected panel size in pixels:", panel_size_pixels)


In [ ]:
image_no = 0

# Load images and preserve 16-bit format
images16bit = [cv2.imread(image_path, cv2.IMREAD_UNCHANGED) for image_path in image_paths]

# Normalize the image 
images16bit_norm = [cv2.normalize(image16bit, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX).astype(np.uint8)
                    for image16bit in images16bit]

images_eq_hist = [cv2.equalizeHist(image16bit_norm) for image16bit_norm in images16bit_norm]

plt.style.use('classic')
fig, ax = plt.subplots(1, 3, figsize=(15, 5))
ax[0].imshow(images16bit[image_no], cmap='gray')
ax[0].set_title('Original image')
ax[0].axis('off')

ax[1].imshow(images16bit_norm[image_no], cmap='gray')
ax[1].set_title('Normalized Image')
ax[1].axis('off')

ax[2].imshow(images_eq_hist[image_no], cmap='gray')
ax[2].set_title('equalized histogram')
ax[2].axis('off')

In [ ]:
image = images_eq_hist[image_no]

variance = filter_by_variance(image)
blur = cv2.medianBlur(variance, 5)
ret, thresh1 = cv2.threshold(blur, 60, 255, cv2.THRESH_BINARY)
masked = np.ma.array(thresh1, mask=thresh1, fill_value=0)

plt.style.use('classic')
fig, ax = plt.subplots(1, 3, figsize=(15, 5))

ax[0].imshow(blur, cmap='gray')
ax[0].set_title('Variance after Median Blur')
ax[0].axis('off')

ax[1].imshow(thresh1, cmap='gray')
ax[1].set_title('Thresholded Image')
ax[1].axis('off')

ax[2].imshow(masked, cmap='gray')
ax[2].set_title('Masked Image')
ax[2].axis('off')

plt.show()

In [ ]:
# This Block can be deleted later in not required anymore

# Ensure masked image is correctly obtained as a binary mask
binary_mask = np.where(masked.mask, 0, 1).astype(np.uint8) * 255

# Apply morphological operations to clean up the binary mask
kernel = np.ones((5, 5), np.uint8)
binary_mask = cv2.morphologyEx(binary_mask, cv2.MORPH_OPEN, kernel)
binary_mask = cv2.morphologyEx(binary_mask, cv2.MORPH_CLOSE, kernel)

# Find contours in the binary mask
contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Filter contours based on size (e.g., minimum area) and solidity
min_contour_area = (panel_size_pixels[0] * panel_size_pixels[1]) * 0.6  # Set to 60% of panels size
min_solidity = 0.75  # Set a threshold for solidity to filter out non-homogeneous areas
filtered_contours = [c for c in contours if
                     cv2.contourArea(c) > min_contour_area and contour_solidity(c) > min_solidity]

# Draw rotated bounding boxes on the original image (for visualization)
image_with_bboxes = cv2.cvtColor(images16bit_norm[image_no],
                                 cv2.COLOR_GRAY2BGR)  # Convert to BGR for colored bounding boxes 

# List to hold bounding box coordinates
bounding_boxes = []

scale_factor = 0.8  # Scale down the bounding box size by 20%

for contour in filtered_contours:
    # Get the minimum area rectangle for the contour
    rect = cv2.minAreaRect(contour)
    box = cv2.boxPoints(rect)  # Get the four points of the rectangle
    box = np.intp(box)  # Convert to integer values

    # Scale down the bounding box
    scaled_box = scale_box(box, scale_factor)
    scaled_box = np.intp(scaled_box)  # Convert to integer values

    cv2.drawContours(image_with_bboxes, [scaled_box], 0, (255, 0, 0), 3)  # Draw the rectangle

    # Save the bounding box coordinates
    bounding_box = {
        "contour_index": len(bounding_boxes),  # Index of the contour
        "coordinates": scaled_box.tolist()  # Convert numpy array to list
    }
    bounding_boxes.append(bounding_box)

# Save bounding box coordinates to a JSON file
with open("bounding_boxes.json", "w") as f:
    json.dump(bounding_boxes, f, indent=4)

# Visualize the result
fig, ax = plt.subplots(1, 3, figsize=(15, 5))

ax[0].imshow(image, cmap='gray')
ax[0].set_title('Equalized Histogram Image')
ax[0].axis('off')

ax[1].imshow(binary_mask, cmap='gray')
ax[1].set_title('Binary Mask with Morphological Operations')
ax[1].axis('off')

ax[2].imshow(image_with_bboxes)
ax[2].set_title('Image with Scaled Bounding Boxes')
ax[2].axis('off')

plt.show()

In [ ]:
# This Block can be deleted later in not required anymore

plt.imshow(image_with_bboxes, cmap='gray')
plt.title(f'IMG_0040_{image_no} with Scaled Bounding Boxes ')
plt.axis('off')

In [ ]:
def split_bounding_box(box, expected_aspect_ratio, threshold=0.1, spacing=10):
    """
    Split the bounding box if its aspect ratio deviates significantly from the expected aspect ratio,
    and space apart the resulting boxes by a specified amount.
    
    Parameters:
        box (ndarray): Array of bounding box points.
        expected_aspect_ratio (float): The expected aspect ratio of the panels.
        threshold (float): The acceptable threshold for aspect ratio deviation.
        spacing (int): The amount of spacing to add between the resulting boxes.
    
    Returns:
        list: A list of bounding boxes (ndarray).
    """
    rect = cv2.minAreaRect(box)
    width, height = rect[1]

    if width == 0 or height == 0:
        return [box]

    # Calculate the aspect ratio of the bounding box
    aspect_ratio = width / height if width > height else height / width

    # Compare aspect ratio with expected aspect ratio
    if abs(aspect_ratio - expected_aspect_ratio) / expected_aspect_ratio > threshold:
        # Determine if width or height should be split
        if width > height:
            new_width = width / 2
            size1 = (new_width, height)
            size2 = (new_width, height)
            offset = (spacing / 2) * np.array([np.cos(np.deg2rad(rect[2])), np.sin(np.deg2rad(rect[2]))])
            center1 = (rect[0][0] - new_width / 2 * np.cos(np.deg2rad(rect[2])),
                       rect[0][1] - new_width / 2 * np.sin(np.deg2rad(rect[2]))) - offset
            center2 = (rect[0][0] + new_width / 2 * np.cos(np.deg2rad(rect[2])),
                       rect[0][1] + new_width / 2 * np.sin(np.deg2rad(rect[2]))) + offset
        else:
            new_height = height / 2
            size1 = (width, new_height)
            size2 = (width, new_height)
            offset = (spacing / 2) * np.array([np.sin(np.deg2rad(rect[2])), -np.cos(np.deg2rad(rect[2]))])
            center1 = (rect[0][0] - new_height / 2 * np.sin(np.deg2rad(rect[2])),
                       rect[0][1] + new_height / 2 * np.cos(np.deg2rad(rect[2]))) - offset
            center2 = (rect[0][0] + new_height / 2 * np.sin(np.deg2rad(rect[2])),
                       rect[0][1] - new_height / 2 * np.cos(np.deg2rad(rect[2]))) + offset

        # Create two new bounding boxes
        box1 = cv2.boxPoints((center1, size1, rect[2]))
        box2 = cv2.boxPoints((center2, size2, rect[2]))

        return [np.intp(box1), np.intp(box2)]

    return [box]

In [ ]:
# Ensure masked image is correctly obtained as a binary mask
binary_mask = np.where(masked.mask, 0, 1).astype(np.uint8) * 255

# Apply morphological operations to clean up the binary mask
kernel = np.ones((5, 5), np.uint8)
binary_mask = cv2.morphologyEx(binary_mask, cv2.MORPH_OPEN, kernel)
binary_mask = cv2.morphologyEx(binary_mask, cv2.MORPH_CLOSE, kernel)

# Find contours in the binary mask
contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)


# Function to calculate the solidity of a contour
def contour_solidity(contour):
    area = cv2.contourArea(contour)
    hull = cv2.convexHull(contour)
    hull_area = cv2.contourArea(hull)
    if hull_area == 0:
        return 0
    return float(area) / hull_area


# Filter contours based on size (e.g., minimum area) and solidity
min_contour_area = (panel_size_pixels[0] * panel_size_pixels[1]) * 0.6  # Set this based on your specific needs
min_solidity = 0.75  # Set a threshold for solidity to filter out non-homogeneous areas
filtered_contours = [c for c in contours if
                     cv2.contourArea(c) > min_contour_area and contour_solidity(c) > min_solidity]

# Expected aspect ratio based on physical panel size
expected_aspect_ratio = physical_panel_size[0] / physical_panel_size[1]
threshold = 0.1  # Threshold for aspect ratio deviation
spacing = 20  # Amount of spacing to add between the resulting boxes

# Draw rotated bounding boxes on the original image (for visualization)
image_with_bboxes = cv2.cvtColor(images16bit_norm[image_no],
                                 cv2.COLOR_GRAY2BGR)  # Convert to BGR for colored bounding boxes

# List to hold bounding box coordinates
bounding_boxes = []

for contour in filtered_contours:
    # Get the minimum area rectangle for the contour
    rect = cv2.minAreaRect(contour)
    box = cv2.boxPoints(rect)  # Get the four points of the rectangle
    box = np.intp(box)  # Convert to integer values

    # Scale down the bounding box
    scaled_box = scale_box(box, scale_factor)
    scaled_box = np.intp(scaled_box)  # Convert to integer values

    # Split bounding box if necessary and space apart
    boxes = split_bounding_box(scaled_box, expected_aspect_ratio, threshold, spacing)

    for bbox in boxes:
        cv2.drawContours(image_with_bboxes, [bbox], 0, (255, 0, 0), 3)  # Draw the rectangle
        # Save the bounding box coordinates
        bounding_box = {
            "contour_index": len(bounding_boxes),  # Index of the contour
            "coordinates": bbox.tolist()  # Convert numpy array to list
        }
        bounding_boxes.append(bounding_box)

# Save bounding box coordinates to a JSON file
with open("bounding_boxes.json", "w") as f:
    json.dump(bounding_boxes, f, indent=4)

# Visualize the result
fig, ax = plt.subplots(1, 3, figsize=(15, 5))

ax[0].imshow(image, cmap='gray')
ax[0].set_title('Equalized Histogram Image')
ax[0].axis('off')

ax[1].imshow(binary_mask, cmap='gray')
ax[1].set_title('Binary Mask with Morphological Operations')
ax[1].axis('off')

ax[2].imshow(image_with_bboxes)
ax[2].set_title('Image with Scaled and Split Bounding Boxes')
ax[2].axis('off')

plt.show()

In [ ]:
plt.imshow(image)

In [ ]:
plt.imshow(binary_mask, cmap='gray')

In [ ]:
plt.imshow(image_with_bboxes, cmap='gray')
plt.title(f'IMG_0040_{image_no} with Scaled Bounding Boxes ')
plt.axis('off')